# <center> Eighth Exercise </center>

### Importing third-party libraries

In [61]:
import sqlite3, sys
from sqlite3 import Error
from shutil import copyfile

!{sys.executable} -m pip install Faker
from faker import Faker


You should consider upgrading via the 'C:\Users\stefan.kolev\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


### Create database

In [62]:
def dbConnection(fileName: str) -> sqlite3.Connection:
    connection = None
    try:
        connection = sqlite3.connect(fileName)
        return connection
    except Error as err:
        print(err)

    return connection
    
connection = dbConnection('test.db')

### First exercise

In [63]:
try:
    cursor = connection.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS company (
        id INTEGER PRIMARY KEY,
        name VARCHAR NOT NULL,
        age INTEGER,
        address VARCHAR,
        salary REAL
    );''')

    ###############
    # Insert data #
    ###############
    for _ in range(10):
        seeder = Faker()
        cursor.execute('INSERT INTO company(name, age, address, salary) VALUES(?, ?, ?, ?)', (seeder.name(), seeder.random_int(1, 65), seeder.address(), seeder.pyfloat(positive=True)))

    connection.commit()
    
    ###############
    # Select data #
    ###############
    employees = cursor.execute('SELECT * FROM company').fetchall()
    print([employee for employee in employees])
    
    ###############
    # Update data #
    ###############
    print(cursor.execute('SELECT * FROM company where id = ?', (1, )).fetchone())

    cursor.execute('UPDATE company SET salary = ? WHERE id = ?', (seeder.pyfloat(positive=True), 1))

    # Select updated record
    print(cursor.execute('SELECT * FROM company where id = ?', (1, ) ).fetchone())    

    ###############
    # Update data #
    ###############
     # Select record before update
    print(cursor.execute('SELECT * FROM company where id = ?', (4, )).fetchone())

    cursor.execute('UPDATE company SET address = ? WHERE id = ?', (seeder.address(), 4))

    # Select updated record
    print(cursor.execute('SELECT * FROM company where id = ?', (4, )).fetchone())

    ###############
    # Delete data #
    ###############
    cursor.execute('DELETE FROM company where id = ?', (2, ))
    connection.commit()

    ###################
    # Statistics data #
    ###################
    minSalary, avgSalary, maxSalary = cursor.execute('SELECT MIN(salary) AS minSalary, AVG(salary) AS avgSalary, MAX(salary) AS maxSalary FROM company').fetchone()

    print(f'\n\nMininum Salary: {minSalary} -- Average Salary: {avgSalary} -- Maximum Salary: {maxSalary}\n\n')
    print(cursor.execute('SELECT * FROM company WHERE salary > ?', (avgSalary,)).fetchall())
except Error as err:
    print(err)

[(1, 'Kurt Rodriguez', 13, '10650 Elizabeth Drive\nSparksmouth, OR 72991', 8.34014455000898), (3, 'Eric Roy', 47, '0404 Smith Square Apt. 917\nRodriguezfurt, WV 40818', 3.6228585123926), (4, 'Amanda Martin', 46, '09620 Bender Fort Suite 381\nParkerborough, DC 53697', 4749850481199.0), (5, 'Stephanie Cuevas', 20, '75222 Candace Trace Suite 787\nGallagherview, NJ 04891', 60795023163.2349), (6, 'Alexander Singleton', 30, '692 Jennifer Via\nTammyshire, HI 81468', 9270969215.63091), (7, 'Tim Richardson', 19, '39094 Chelsea Junction\nEast James, RI 95923', 90052.496806958), (8, 'Megan Bowen', 60, '41313 Cross Pike Suite 585\nNorth Dan, CT 21133', 9620666.10952312), (9, 'John Greer', 63, '888 Humphrey Mills\nNorth Jessicaton, MI 29238', 36609864070.1546), (10, 'Glenda Bennett', 53, '6758 John Canyon Apt. 130\nYoungbury, ME 02051', 32.41581924284), (11, 'Dr. Troy Black MD', 47, '70176 Lynch Circles\nRichardsonview, WI 94768', 246569874308.768), (12, 'Karen Dixon', 43, 'PSC 7570, Box 0135\nAPO 

### Second Exercise

In [64]:
try:
    cursor = connection.cursor()

    cursor.execute('DROP TABLE groups')
    cursor.execute('DROP TABLE contacts')
    cursor.execute('DROP TABLE contacts_groups')

    # Create Tables
    cursor.execute('''CREATE TABLE IF NOT EXISTS groups (
        id INTEGER PRIMARY KEY,
        name VARCHAR NOT NULL
    );''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS contacts (
        id INTEGER PRIMARY KEY,
        firstName VARCHAR NOT NULL,
        lastName VARCHAR NOT NULL,
        email VARCHAR NOT NULL UNIQUE,
        phone VARCHAR NOT NULL UNIQUE,
        age INTEGER,
        sex VARCHAR NOT NULL
    );''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS contacts_groups (
        id INTEGER PRIMARY KEY,
        contact_id INTEGER NOT NULL,
        group_id INTEGER NOT NULL,
        FOREIGN KEY(contact_id) REFERENCES contacts(id)
        FOREIGN KEY(group_id) REFERENCES groups(id)
    );''')

    ###############
    # Insert data #
    ###############
    for _ in range(3):
        seeder = Faker()
        cursor.execute('INSERT INTO groups(name) VALUES(?)', (seeder.job(), ))

    for _ in range(10):
        seeder = Faker()
        cursor.execute('INSERT INTO contacts(firstName, lastName, email, phone, age, sex) VALUES(?, ?, ?, ?, ?, ?)', (seeder.first_name(), seeder.last_name(), seeder.email(), seeder.phone_number(), seeder.random_int(1, 65), 'F' if seeder.random_int(0, 1) == 0 else 'M'))

    connection.commit()
    
    ###############
    # Select data #
    ###############
    groups = cursor.execute('SELECT * FROM groups').fetchall()
    print([group for group in groups])

    contacts = cursor.execute('SELECT * FROM contacts').fetchall()
    print([contact for contact in contacts])

    ###############
    # Insert data #
    ###############
    for index in range(len(contacts)):
        cursor.execute('INSERT INTO contacts_groups(contact_id, group_id) VALUES(?, ?)', (seeder.random_int(1, len(contacts)), seeder.random_int(1, len(groups))))

    connection.commit()
    
    contacts_groups = cursor.execute('SELECT * FROM contacts_groups').fetchall()
    print([contact_group for contact_group in contacts_groups])

    youngestContact = cursor.execute('''
    SELECT baseContacts.*, MIN(baseContacts.age)
    FROM contacts_groups 
    LEFT JOIN contacts as baseContacts
    ON contacts_groups.contact_id = baseContacts.id WHERE group_id = ? AND baseContacts.age
    ''', (2, )).fetchone()

    oldestContact = cursor.execute('''
    SELECT baseContacts.*, MAX(baseContacts.age)
    FROM contacts_groups 
    LEFT JOIN contacts as baseContacts
    ON contacts_groups.contact_id = baseContacts.id WHERE group_id = ?
    ''', (2, )).fetchone()

    print(youngestContact)
    print(oldestContact)

    contactByPhone = cursor.execute('''
    SELECT *
    FROM contacts
    WHERE phone LIKE ? 
    ''', ('%9%', )).fetchone()

    print(contactByPhone)
    ###############
    # Update data #
    ###############
    print(cursor.execute('SELECT * FROM contacts where id = ?', (5, )).fetchone())

    cursor.execute('UPDATE contacts SET phone = ? WHERE id = ?', (seeder.phone_number(), 5))

    # Select updated record
    print(cursor.execute('SELECT * FROM contacts where id = ?', (5, ) ).fetchone())    

    ###############
    # Delete data #
    ###############
    cursor.execute('DELETE FROM contacts where id = ?', (7, ))
    connection.commit()

    contacts = cursor.execute('SELECT * FROM contacts').fetchall()
    print([contact for contact in contacts])
except Error as err:
    print(err)

[(1, 'Public house manager'), (2, 'Editor, commissioning'), (3, 'Veterinary surgeon')]
[(1, 'Patricia', 'Mccarthy', 'bryanmacdonald@anderson-oliver.org', '836-294-5667x9408', 49, 'M'), (2, 'Michael', 'Gutierrez', 'jessica61@huang-baker.org', '+1-123-812-4813x511', 7, 'F'), (3, 'Amanda', 'Sullivan', 'holly28@graham.com', '665-927-3373x1247', 33, 'M'), (4, 'Donald', 'Ramirez', 'timothy57@hotmail.com', '001-622-353-5880x927', 50, 'M'), (5, 'Timothy', 'Mueller', 'njohnston@hotmail.com', '4061011621', 42, 'F'), (6, 'Ricardo', 'Mosley', 'antoniogonzalez@ellis-caldwell.org', '721.275.3969x82547', 30, 'F'), (7, 'William', 'Howard', 'yhammond@johnson.com', '382-886-7768', 38, 'M'), (8, 'Levi', 'Mercado', 'william84@hotmail.com', '987-055-3474', 49, 'F'), (9, 'John', 'Reynolds', 'brandonthomas@hotmail.com', '001-952-425-2304x39437', 32, 'F'), (10, 'Danielle', 'Mcneil', 'traneric@yahoo.com', '001-639-975-2087x414', 25, 'F')]
[(1, 9, 3), (2, 4, 3), (3, 1, 2), (4, 6, 2), (5, 5, 2), (6, 2, 1), (7, 7

### Close database connection

In [65]:
connection.close()

## <center> Extra Exercises </center>

### First Exercise

In [66]:
conn = dbConnection('extras.db')

try:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS exercise (
        id INTEGER PRIMARY KEY,
        name VARCHAR NOT NULL
    );''')

    for _ in range(10):
        faker = Faker()
        cursor.execute('INSERT INTO exercise(name) VALUES(?)', (faker.name(),))
        
    print(cursor.execute('SELECT * FROM exercise').fetchall())
except Error as err:
    print(err)


database is locked


### Second Exercise

In [67]:
examples = []

for _ in range(10):
    seeder = Faker()
    examples.append(seeder.name())

for element in examples:
    cursor.execute('INSERT INTO exercise(name) VALUES(?)', (element,))

print('List: ' + ' -- '.join(examples))
print('Database rows: ' + str(cursor.execute('SELECT * FROM exercise').fetchall()))

OperationalError: database is locked

### Third Exercise

In [45]:
for element in examples:
    cursor.execute('INSERT INTO exercise(name) VALUES(?)', (input(), ))

print('Database rows: ' + str(cursor.execute('SELECT * FROM exercise').fetchall()))

Database rows: [(1, 'Sierra George'), (2, 'Rachel Salinas'), (3, 'William Wheeler'), (4, 'Brittany Cox'), (5, 'Lisa Solis'), (6, 'Jonathan Conner'), (7, 'Susan Garcia'), (8, 'Bruce Hicks'), (9, 'Meredith Garcia'), (10, 'Lori Russell'), (11, 'Ronald Wells'), (12, 'Robyn Fuentes'), (13, 'Karen Clarke'), (14, 'Nicholas Ward'), (15, 'Tracey Frazier'), (16, 'Kristen Thomas'), (17, 'Jeremy Wallace'), (18, 'Christy Hicks'), (19, 'Leslie Gibson'), (20, 'Marcus Garrett'), (21, ''), (22, ''), (23, ''), (24, ''), (25, ''), (26, ''), (27, ''), (28, ''), (29, ''), (30, '')]


### Fourth Exercise

In [46]:
print(cursor.execute('SELECT COUNT(id) as rowCount FROM exercise').fetchone())

(30,)


### Fifth Exercise

In [47]:
# Record ID
recordId = 4

# Select record before update
print(f'\n\nRecords before update:\n\n' + str(cursor.execute('SELECT * FROM exercise').fetchall()))

print(f'\n\nUpdating record with id {recordId}...')
cursor.execute('UPDATE exercise SET name = ? WHERE id = ?', (Faker().name(), recordId))

# Select updated record
print(f'\n\nRecords after update:\n\n' + str(cursor.execute('SELECT * FROM exercise').fetchall()))



Records before update:

[(1, 'Sierra George'), (2, 'Rachel Salinas'), (3, 'William Wheeler'), (4, 'Brittany Cox'), (5, 'Lisa Solis'), (6, 'Jonathan Conner'), (7, 'Susan Garcia'), (8, 'Bruce Hicks'), (9, 'Meredith Garcia'), (10, 'Lori Russell'), (11, 'Ronald Wells'), (12, 'Robyn Fuentes'), (13, 'Karen Clarke'), (14, 'Nicholas Ward'), (15, 'Tracey Frazier'), (16, 'Kristen Thomas'), (17, 'Jeremy Wallace'), (18, 'Christy Hicks'), (19, 'Leslie Gibson'), (20, 'Marcus Garrett'), (21, ''), (22, ''), (23, ''), (24, ''), (25, ''), (26, ''), (27, ''), (28, ''), (29, ''), (30, '')]


Updating record with id 4...


Records after update:

[(1, 'Sierra George'), (2, 'Rachel Salinas'), (3, 'William Wheeler'), (4, 'Terry White'), (5, 'Lisa Solis'), (6, 'Jonathan Conner'), (7, 'Susan Garcia'), (8, 'Bruce Hicks'), (9, 'Meredith Garcia'), (10, 'Lori Russell'), (11, 'Ronald Wells'), (12, 'Robyn Fuentes'), (13, 'Karen Clarke'), (14, 'Nicholas Ward'), (15, 'Tracey Frazier'), (16, 'Kristen Thomas'), (17, 'J

### Sixth Exercise

In [48]:
print(f'Updating all records...')
cursor.execute('UPDATE exercise SET name = ?', ('asdasd',))

# Select updated record
print(f'\n\nRecords after update:\n\n' + str(cursor.execute('SELECT * FROM exercise').fetchall()))

Updating all records...


Records after update:

[(1, 'asdasd'), (2, 'asdasd'), (3, 'asdasd'), (4, 'asdasd'), (5, 'asdasd'), (6, 'asdasd'), (7, 'asdasd'), (8, 'asdasd'), (9, 'asdasd'), (10, 'asdasd'), (11, 'asdasd'), (12, 'asdasd'), (13, 'asdasd'), (14, 'asdasd'), (15, 'asdasd'), (16, 'asdasd'), (17, 'asdasd'), (18, 'asdasd'), (19, 'asdasd'), (20, 'asdasd'), (21, 'asdasd'), (22, 'asdasd'), (23, 'asdasd'), (24, 'asdasd'), (25, 'asdasd'), (26, 'asdasd'), (27, 'asdasd'), (28, 'asdasd'), (29, 'asdasd'), (30, 'asdasd')]


### Seventh Exercise

In [49]:
cursor.execute('DELETE FROM exercise WHERE id = 28')

# Select all records
print(f'\n\nRecords after delete:\n\n' + str(cursor.execute('SELECT * FROM exercise').fetchall()))



Records after delete:

[(1, 'asdasd'), (2, 'asdasd'), (3, 'asdasd'), (4, 'asdasd'), (5, 'asdasd'), (6, 'asdasd'), (7, 'asdasd'), (8, 'asdasd'), (9, 'asdasd'), (10, 'asdasd'), (11, 'asdasd'), (12, 'asdasd'), (13, 'asdasd'), (14, 'asdasd'), (15, 'asdasd'), (16, 'asdasd'), (17, 'asdasd'), (18, 'asdasd'), (19, 'asdasd'), (20, 'asdasd'), (21, 'asdasd'), (22, 'asdasd'), (23, 'asdasd'), (24, 'asdasd'), (25, 'asdasd'), (26, 'asdasd'), (27, 'asdasd'), (29, 'asdasd'), (30, 'asdasd')]


### Eighth Exercise

In [50]:
copyfile('test.db', 'test.backup.db')

'test.backup.db'

### Connection close

In [51]:
# conn.close()